In [1]:
# Package to create synthetic data for causal inference
from dowhy import datasets
# Data processing
import pandas as pd
import numpy as np
# Causal inference
from causalinference import CausalModel

In [2]:
# Set random seed
np.random.seed(42)
# Create a synthetic dataset
data = datasets.linear_dataset(
    beta=10,
    num_common_causes=4,
    num_samples=10000,
    treatment_is_binary=True,
    outcome_is_binary=False)
# Create Dataframe
df = data['df']
# Take a look at the data
df.head()

,W0,W1,W2,W3,v0,y
0,-0.485073,0.667292,2.043201,0.964752,True,17.107666
1,-0.720394,1.443989,0.000570,-0.268413,True,9.638488
2,-0.008957,-1.011852,-1.260930,-0.364971,False,-5.821974
3,-1.263751,1.215676,-0.444036,-1.214987,True,3.508742
4,1.214729,0.675652,0.531516,-1.227431,True,12.367930


In [3]:
# Rename columns
df = df.rename({'v0': 'treatment', 'y': 'outcome'}, axis=1)
# Create the treatment variable, and change boolean values to 1 and 0
df['treatment'] = df['treatment'].apply(lambda x: 1 if x == True else 0)
# Take a look at the data
df.head()

,W0,W1,W2,W3,treatment,outcome
0,-0.485073,0.667292,2.043201,0.964752,1,17.107666
1,-0.720394,1.443989,0.000570,-0.268413,1,9.638488
2,-0.008957,-1.011852,-1.260930,-0.364971,0,-5.821974
3,-1.263751,1.215676,-0.444036,-1.214987,1,3.508742
4,1.214729,0.675652,0.531516,-1.227431,1,12.367930


In [4]:
# Run causal model
causal = CausalModel(Y = df['outcome'].values, D = df['treatment'].values, X = df[['W0', 'W1', 'W2', 'W3']].values)
# Print summary statistics
print(causal.summary_stats)


Summary Statistics

                      Controls (N_c=2269)        Treated (N_t=7731)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y       -2.191        4.217       13.940        4.842       16.132

                      Controls (N_c=2269)        Treated (N_t=7731)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0       -0.779        0.939       -0.088        0.971        0.724
             X1       -0.181        0.685        1.207        0.829        1.825
             X2        0.122        0.997        0.555        0.991        0.435
             X3        0.127        0.969        0.221        1.005        0.095



In [5]:
# Automated propensity score estimation
causal.est_propensity_s()
# Propensity model results
print(causal.propensity)



Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept     -0.052      0.053     -0.982      0.326     -0.156      0.052
            X1      4.525      0.117     38.816      0.000      4.297      4.754
            X0      2.171      0.067     32.537      0.000      2.040      2.302
            X2      1.350      0.054     25.134      0.000      1.244      1.455
            X3      0.286      0.044      6.500      0.000      0.200      0.373



In [6]:
# Propensity scores
causal.propensity['fitted']


array([0.99295272, 0.99217314, 0.00156753, ..., 0.69143426, 0.99983862,
       0.99943713])

In [7]:
# Trim using the optimal cutoff value
causal.trim_s()
# Check the optimal cutoff value
causal.cutoff

In [8]:
# Print summary statistics
print(causal.summary_stats)



Summary Statistics

                      Controls (N_c=1189)        Treated (N_t=1463)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y       -0.833        3.715       10.188        3.881       11.020

                      Controls (N_c=1189)        Treated (N_t=1463)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0       -0.623        0.914       -0.514        0.916        0.120
             X1        0.121        0.516        0.349        0.527        0.436
             X2        0.240        0.956        0.311        0.960        0.074
             X3        0.133        0.962        0.157        0.990        0.024

